# LLM Iteration 26

**Aim**

Create a method of warm-up analysis using number of patients waiting and mean waiting time.

**Prompt**

Using this WarmupAuditor class, create a warmup_analysis function that runs multiple replications of the model and collects the audit data using the class for each run. Collect the audit results for each replication into a dictionary containing two dataframes, one for number of patients waiting and one for bed waiting time. The dataframes should have the audit number as the index and the replication number as the columns.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**Additional code to run model**

In [1]:
def trace(msg):
    '''
    Utility function for printing simulation.
    Set the TRACE constant to FALSE to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [2]:
TRACE = False

In [3]:
import simpy
import numpy as np
import math

In [4]:
class Scenario:
    def __init__(self, num_beds, run_length=60*2, seed=None):
        self.num_beds = num_beds
        self.run_length = run_length
        self.seed = seed
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self, num_runs):
        interarrival_distributions = {}
        seeds = self.generate_seeds(num_runs)
        for patient_type, seed in seeds.items():
            if seed[0] is not None:
                np.random.seed(seed)
                interarrival_distributions[patient_type] = np.random.\
                    default_rng(seed)
            else:
                interarrival_distributions[patient_type] = np.random.\
                    default_rng()
        return interarrival_distributions

    def generate_seeds(self, num_runs):
        seeds = {}
        for patient_type, seed in self.random_seeds.items():
            if self.seed is not None:
                seeds[patient_type] = [seed + i + self.seed for i in \
                                       range(num_runs)]
            else:
                seeds[patient_type] = [None for i in range(num_runs)]
        return seeds

In [5]:
class StrokeWard:
    def __init__(self, scenario):
        self.env = simpy.Environment()
        self.beds = simpy.Resource(self.env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.\
            create_distributions(num_runs=1)
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.patient_type_store = []
        self.los_store = []

    def update_length_of_stay(self, patient_type, destination):
        if patient_type == 'Stroke':
            if destination == 'ESD':
                return {"mean": 4.6, "std_dev": 4.8}
            elif destination == 'Other':
                # Mortality rate for Stroke patients going to Other
                if np.random.rand() < 0.02:
                     # Adjusted parameters for mortality case
                    return {"mean": 7.0, "std_dev": 8.7}
        return self.length_of_stay[patient_type]

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))

            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())

            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)

            length_of_stay_params = self.update_length_of_stay(patient.\
                                                               patient_type,
                                                               route_choice)

            mu, sigma = calculate_mu_sigma(length_of_stay_params["mean"],
                                           length_of_stay_params["std_dev"])

            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))

            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].exponential(
                        scenario.default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)

        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }

        summary_df = pd.DataFrame(summary_data)

        print("Summary:")
        print(summary_df)

In [6]:
class WarmupAuditor:
    def __init__(self, scenario, audit_interval):
        self.scenario = scenario
        self.audit_interval = audit_interval
        self.audit_times = []
        self.waiting_patients_audit = []
        self.wait_times_audit = []

    def run_model(self):
        stroke_ward = StrokeWard(self.scenario)
        env = stroke_ward.env

        PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
        for patient_type in PATIENTS:
            env.process(stroke_ward.patient_arrival(patient_type))

        env.process(self.audit_model(stroke_ward))
        env.run(until=self.scenario.run_length)

        return self.summary()

    def audit_model(self, stroke_ward):
        while stroke_ward.env.now < self.scenario.run_length:
            yield stroke_ward.env.timeout(self.audit_interval)
            self.audit_times.append(stroke_ward.env.now)
            self.waiting_patients_audit.append(np.mean(stroke_ward.\
                                                       waiting_patients))
            self.wait_times_audit.append(np.mean(stroke_ward.wait_times))

    def summary(self):
        audit_data = {
            'Audit Time': self.audit_times,
            'Mean Waiting Patients': self.waiting_patients_audit,
            'Mean Wait Time': self.wait_times_audit
        }

        summary_df = pd.DataFrame(audit_data)

        return summary_df

In [11]:
def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

**LLM Output**

To create a warmup_analysis function that runs multiple replications of the model and collects the audit data using the WarmupAuditor class for each run, and then stores the audit results in a dictionary containing two DataFrames (one for the number of patients waiting and one for bed waiting time), you can use the following code:

In [8]:
import pandas as pd

In [9]:
def warmup_analysis(scenario, num_replications, audit_interval):
    waiting_patients_results = {}
    wait_times_results = {}

    for i in range(num_replications):
        warmup_auditor = WarmupAuditor(scenario, audit_interval)
        audit_results = warmup_auditor.run_model()

        waiting_patients_results[f'Replication {i+1}'] = audit_results\
            ['Mean Waiting Patients']
        wait_times_results[f'Replication {i+1}'] = audit_results\
            ['Mean Wait Time']

    waiting_patients_df = pd.DataFrame(waiting_patients_results)
    wait_times_df = pd.DataFrame(wait_times_results)

    waiting_patients_df = waiting_patients_df.set_index(audit_results\
                                                        ['Audit Time'])
    wait_times_df = wait_times_df.set_index(audit_results['Audit Time'])

    results = {
        'Waiting Patients': waiting_patients_df,
        'Wait Times': wait_times_df
    }

    return results

In [10]:
# Example usage:
scenario = Scenario(num_beds=10, seed=42) ### MODIFICATION - added seed ###
num_replications = 5
audit_interval = 5

warmup_results = warmup_analysis(scenario, num_replications, audit_interval)

print("Waiting Patients:")
print(warmup_results['Waiting Patients'])
print("\nWait Times:")
print(warmup_results['Wait Times'])

Waiting Patients:
            Replication 1  Replication 2  Replication 3  Replication 4  \
Audit Time                                                               
5                0.000000       0.000000       0.000000       0.000000   
10               0.000000       0.000000       0.000000       0.000000   
15               0.000000       0.000000       0.000000       0.000000   
20               0.000000       0.000000       0.000000       0.000000   
25               0.000000       0.000000       0.000000       0.000000   
30               0.000000       0.000000       0.000000       0.000000   
35               0.000000       0.000000       0.000000       0.000000   
40               0.017857       0.017857       0.017857       0.017857   
45               0.031250       0.031250       0.031250       0.031250   
50               0.773333       0.773333       0.773333       0.773333   
55               1.144578       1.144578       1.144578       1.144578   
60               1.2

In this code:
- The warmup_analysis function takes the scenario, num_replications, and audit_interval as input parameters.
Two dictionaries, waiting_patients_results and wait_times_results, are created to store the audit results for each replication.
- Inside the loop, a WarmupAuditor instance is created for each replication, and the run_model method is called to obtain the audit results.
- The audit results for the mean number of waiting patients and mean wait times are stored in the respective dictionaries, with the replication number as the key.
- After the loop, two DataFrames are created from the dictionaries, with the audit times as the index and the replication numbers as the columns.
- The resulting DataFrames are stored in a dictionary, results, with keys 'Waiting Patients' and 'Wait Times'.
- The results dictionary is returned by the warmup_analysis function.
- In the example usage, the warmup_analysis function is called with the specified scenario, number of replications, and audit interval.
- The resulting DataFrames for waiting patients and wait times are printed.

With this implementation, the warmup_analysis function runs multiple replications of the model, collects the audit data using the WarmupAuditor class for each run, and stores the audit results in a dictionary containing two DataFrames, one for the number of patients waiting and one for bed waiting time. The DataFrames have the audit number as the index and the replication number as the columns, allowing for easy analysis and visualization of the warmup data across multiple replications.

**Reflection**

The LLM has allowed the the warm-up analysis to be repeated multiple times, but the result is the same each time as the same seed is being used each time. This is the same issue that was found with the multiple replications function.